In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.pardir))
from argparse import ArgumentParser

import torch
from voltaml.compile import VoltaGPUCompiler
from voltaml.inference import gpu_performance
import torchvision

### Load Model 

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_mobilenet_v3_large', pretrained=True)

## Set parameters for FP16

In [ ]:
input_shape = (1,3,224,224)
precision = 'fp16'
compiled_model_dir = 'deeplabv3_mobile.engine' ## Set Model dir
throughput_batch_size = 1

### Compile Model

In [ ]:
compiler = VoltaGPUCompiler(
    model=model,
    output_dir=compiled_model_dir,
    input_shape=input_shape,
    precision=precision,
    simplify=True
)

compiled_model = compiler.compile()

In [ ]:
gpu_performance(compiled_model_dir, model, input_shape=input_shape, throughput_batch_size=throughput_batch_size, is_yolo=False)

### Set parameters for INT8

In [ ]:
input_shape = (1,3,224,224)
precision = 'int8'
compiled_model_dir = 'deeplabv3_mobile_int8.engine' ## Compiled model directory
throughput_batch_size = 1
calib_input = '/workspace/val2017/' ## Calib input images path
calib_cache = '' ## Cache name
calib_num_images=25000
calib_batch_size=8
calib_preprocessor='V1MS'

### Compile Model

In [ ]:
compiler = VoltaGPUCompiler(
    model=model,
    output_dir=compiled_model_dir,
    input_shape=input_shape,
    precision=precision,
    calib_input=calib_input,
    calib_cache=calib_cache,
    calib_num_images=calib_num_images,
    calib_batch_size=calib_batch_size,
    calib_preprocessor=calib_preprocessor
)

compiled_model = compiler.compile()

In [ ]:
gpu_performance(compiled_model_dir, model, input_shape=input_shape, throughput_batch_size=throughput_batch_size, is_yolo=False)